<a href="https://colab.research.google.com/github/dantrainor9/Cervical_cancer_risk_factors/blob/main/Project_2_Part_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
path = '/content/drive/MyDrive/CodingDojo Data Science Bootcamp/07 Week 7 Logistic Regression and Regularization/risk_factors_cervical_cancer.csv'
df = pd.read_csv(path)
df.head() 
#checking the first 5 rows in the dataset to make sure it loaded correctly

In [ ]:
df.shape

Multiple targets are present in the above dataset gathered from the UCI machine learning repository, as multiple diagnostic tools were used. However, as 'Biopsy' is the most common way of diagnosing cancer in the United States, I will be using that as my primary target after comparing the performance of the different diagnostic tests. 
This is a classification problem with 32 features.
There are 858 rows of data. 
I anticipate problems with missing data, as participants were not required to answer every question due to the sensitive nature of the survey.

In [ ]:
df = df.replace('?', np.nan) 
#replacing question marks in the dataset with np.nan so I can impute using SimpleImputer

In [ ]:
df.isna().sum() 
#checking for null values

In [ ]:
X = df[['Age', 
        'Number of sexual partners', 
        'First sexual intercourse', 
        'Num of pregnancies', 
        'Smokes', 
        'Smokes (years)', 
        'Smokes (packs/year)', 
        'Hormonal Contraceptives', 
        'Hormonal Contraceptives (years)', 
        'IUD', 
        'IUD (years)', 
        'STDs', 
        'STDs (number)', 
        'STDs:condylomatosis', 
        'STDs:vaginal condylomatosis', 
        'STDs:vulvo-perineal condylomatosis', 
        'STDs:syphilis', 
        'STDs:pelvic inflammatory disease', 
        'STDs:genital herpes', 
        'STDs:molluscum contagiosum', 
        'STDs:HIV',  
        'STDs:HPV', 
        'STDs: Number of diagnosis', 
        'Dx:Cancer', 
        'Dx:CIN', 
        'Dx:HPV', 
        'Dx']]
y = df['Biopsy']
#excluding 2 columns as features, time since first and last diagnosis of STDs
#excluding 3 columns as all participants answered the same, STDs: cervical condylomatosis, hepatitis B, and AIDS

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
X_train

In [ ]:
mean_impute = SimpleImputer(missing_values=np.nan, strategy='mean')
mode_impute = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
#building a column transformer for imputing

In [ ]:
mode_list = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'Number of sexual partners', 'Num of pregnancies', 'First sexual intercourse',
             'STDs', 'STDs:condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease', 
             'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:HIV',  'STDs:HPV']
#splitting into different lists based on imputation method

In [ ]:
mean_list = ['Smokes (years)', 'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'IUD (years)', 'STDs (number)', 'STDs: Number of diagnosis']

In [ ]:
mode_tuple = (mode_impute, mode_list)
mean_tuple = (mean_impute, mean_list)

In [ ]:
coltrans = make_column_transformer(mode_tuple, mean_tuple, remainder='passthrough')
#instantiating column transformer

In [ ]:
coltrans.fit(X_train)
X_train_imp = coltrans.transform(X_train)
X_test_imp = coltrans.transform(X_test)
#fitting and transforming data

In [ ]:
X_train_imp

In [ ]:
col_names = ['Smokes', 'Hormonal Contraceptives', 'IUD', 'Number of sexual partners', 'Num of pregnancies', 'First sexual intercourse',
             'STDs', 'STDs:condylomatosis', 'STDs:vaginal condylomatosis', 'STDs:vulvo-perineal condylomatosis', 'STDs:syphilis', 'STDs:pelvic inflammatory disease', 
             'STDs:genital herpes', 'STDs:molluscum contagiosum', 'STDs:HIV',  'STDs:HPV', 
             'Smokes (years)', 'Smokes (packs/year)', 'Hormonal Contraceptives (years)', 'IUD (years)', 'STDs (number)', 'STDs: Number of diagnosis', 'Age',
             'Dx:Cancer', 
             'Dx:CIN', 
             'Dx:HPV',
             'Dx']
#column transformer moved around my columns so they are listed here in the revised order for column name assignment in the next step

In [ ]:
X_train_eda = pd.DataFrame(X_train_imp, columns=col_names)
X_test_eda = pd.DataFrame(X_test_imp, columns=col_names)
#reconverting from array to dataframe

In [ ]:
X_train_eda.value_counts()
#verifying all my columns are in the right place

In [ ]:
X_train_eda.info()
#verifying successful imputations of missing values

In [ ]:
X_train_eda = X_train_eda.astype('float')
#messy data types, fixing that

In [ ]:
X_train_eda.info()

In [ ]:
X_test_eda = X_test_eda.astype('float')
#fixing data types in X_test as well

In [ ]:
X_test_eda.info()

In [ ]:
plt.hist(X_train_eda['Age'], bins=30, edgecolor='k')
plt.title('Age');
#looks like an outlier here, at least one much older participant

In [ ]:
plt.hist(X_train_eda['Number of sexual partners'], bins=30, edgecolor='k')
plt.title('Number of Sexual Partners')
plt.xlabel('Number of Sexual Partners')
plt.ylabel('Number of Respondents')
plt.tight_layout()
plt.savefig('NumberofSexualPartners.png', dpi = 300);
#some outliers here as well

In [ ]:
plt.hist(X_train_eda['First sexual intercourse'], bins=20, edgecolor='k')
plt.title('Age of First Sexual Intercourse')
plt.xlabel('Age (years)')
plt.ylabel('Number of Respondents')
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig('AgeofFirstSexualIntercourse.png', dpi = 300);
#fairly normal with a bit of a right skew

In [ ]:
plt.hist(X_train_eda['Num of pregnancies'], bins=8, edgecolor='k')
plt.title('Number of Pregnancies')
plt.xlabel('Number of Pregnancies')
plt.ylabel('Number of Respondents')
plt.tight_layout()
plt.savefig('NumberofPregnancies.png', dpi = 300);
#right skew here

In [ ]:
plt.hist(X_train_eda['Smokes'], bins=2, edgecolor='k')
plt.title('Smokes');

In [ ]:
plt.hist(X_train_eda['Smokes (years)'], bins=15, edgecolor='k')
plt.title('Years of Smoking')
plt.xlabel('Years of Smoking')
plt.ylabel('Number of Respondents')
plt.tight_layout()
plt.savefig('YearsofSmoking.png', dpi=300);
#right skew, though most of these probably count as outliers as most participants were non-smokers

In [ ]:
plt.hist(X_train_eda['Smokes (packs/year)'], bins=15, edgecolor='k')
plt.title('Packs of Cigarettes Smoked per Year');
#similar skew to the graph above

In [ ]:
z = X_train_eda['Hormonal Contraceptives'].value_counts()

a = list(z.index)
b = list(z)

plt.figure(figsize=(14,10))
plt.bar(a, b, edgecolor='k')
plt.title('Hormonal Contraceptives', fontsize=20)
plt.xticks(ticks=[0,1], labels=['No History', 'History of Use'], fontsize=16),
plt.ylabel('Number of Participants', fontsize=16);
#more common for our participants to have taken hormonal contraceptives than not

In [ ]:
plt.figure(figsize=(16,12))
plt.hist(X_train_eda['Hormonal Contraceptives (years)'], bins=20, edgecolor='k')
plt.title('Years of Taking Hormonal Contraceptives', fontsize=20),
plt.xlabel('Number of Years', fontsize=16),
plt.ylabel('Number of Participants', fontsize=16);
#right skew with some outliers

In [ ]:
plt.hist(X_train_eda['IUD'], bins=2, edgecolor='k')
plt.title('IUD');
#IUD is much less common than hormonal contraceptives in our dataset

In [ ]:
plt.hist(X_train_eda['IUD (years)'], bins=20, edgecolor='k')
plt.title('Years with IUD');
#with so few IUD users, most on the tail of this skew are likely outliers

In [ ]:
z = X_train_eda['STDs'].value_counts()

a = list(z.index)
b = list(z)

plt.figure(figsize=(16,12))
plt.bar(a, b, edgecolor='k')
plt.title('STDs', fontsize=20)
plt.xticks(ticks=[0,1], labels=['No History of STDs', 'History of STDs'], fontsize=16)
plt.ylabel('Number of Participants', fontsize=16);
#STDs are uncommon it seems, or at least not commonly admitted to

In [ ]:
plt.hist(X_train_eda['STDs (number)'], bins=5, edgecolor='k')
plt.title('Number of STDs');
#also more common to disclose fewer than 3

In [ ]:
plt.hist(X_train_eda['STDs:condylomatosis'], bins=2, edgecolor='k')
plt.title('Condylomatosis');

In [ ]:
plt.hist(X_train_eda['STDs:vaginal condylomatosis'], bins=2, edgecolor='k')
plt.title('Vaginal Condylomatosis');

In [ ]:
plt.hist(X_train_eda['STDs:vulvo-perineal condylomatosis'], bins=2, edgecolor='k')
plt.title('Vulvo-Perineal Condylomatosis');

In [ ]:
plt.hist(X_train_eda['STDs:syphilis'], bins=2, edgecolor='k')
plt.title('Syphilis');

In [ ]:
plt.hist(X_train_eda['STDs:pelvic inflammatory disease'], bins=2, edgecolor='k')
plt.title('Pelvic Inflammatory Disease');

In [ ]:
plt.hist(X_train_eda['STDs:genital herpes'], bins=2, edgecolor='k')
plt.title('Genital Herpes');

In [ ]:
plt.hist(X_train_eda['STDs:molluscum contagiosum'], bins=2, edgecolor='k')
plt.title('Molluscum Contagiosum');

In [ ]:
plt.hist(X_train_eda['STDs:HIV'], bins=2, edgecolor='k')
plt.title('HIV');
#still some with HIV, however, this does not necessarily confound their data

In [ ]:
z = X_train_eda['STDs:HPV'].value_counts()

a = list(z.index)
b = list(z)

plt.figure(figsize=(16,12))
plt.bar(a, b, edgecolor='k')
plt.title('HPV', fontsize=20)
plt.xticks(ticks=[0,1], labels=['No History of HPV', 'History of HPV'], fontsize=16)
plt.ylabel('Number of Participants', fontsize=16);

In [ ]:
plt.hist(X_train_eda['STDs: Number of diagnosis'], bins=4, edgecolor='k')
plt.title('Number of STD diagnoses');
#looks like anyone who answered more than 1 is an outlier in this column

In [ ]:
plt.hist(X_train_eda['Dx:Cancer'], bins=2, edgecolor='k')
plt.title('Diagnosis of Cancer');

In [ ]:
plt.hist(X_train_eda['Dx:CIN'], bins=2, edgecolor='k')
plt.title('Diagnosis of CIN');

In [ ]:
plt.hist(X_train_eda['Dx:HPV'], bins=2, edgecolor='k')
plt.title('Diagnosis of HPV');

In [ ]:
plt.hist(X_train_eda['Dx'], bins=2, edgecolor='k')
plt.title('Previous Diagnosis');

In [ ]:
plt.hist(y_train, bins=2, edgecolor='k')
plt.title('Positive Biopsy (Target Column)');
#target column

In [ ]:
X_train_eda['Biopsy'] = y_train
#re-adding the target column for checking correlations

In [ ]:
train_corr = X_train_eda.corr()

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(data=train_corr, cmap='Greens', annot=True);

Unfortunately no strong correlations with the Biopsy column. This will make learning harder for whatever model I test.

In [ ]:
plt.figure(figsize=(30,30))
sns.pairplot(X_train_eda);

In [ ]:
X_train_eda = X_train_eda.drop(columns=['Biopsy'])
#re-dropping target column

In [ ]:
scaler = StandardScaler()
#instantiating scaler

In [ ]:
xgb = XGBClassifier()
#instantiating xgb model

In [ ]:
xgb_pipe = make_pipeline(scaler, xgb)

In [ ]:
xgb_pipe.fit(X_train_eda, y_train)

In [ ]:
xgb_preds = xgb_pipe.predict(X_test)

In [ ]:
print(classification_report(y_test,xgb_preds))
#accuracy is fine, but this model is completely ignoring the positive class
#this may be from imbalances in the dataset

In [ ]:
rf = RandomForestClassifier()

In [ ]:
rf_pipe = make_pipeline(scaler,rf)

In [ ]:
rf_pipe.fit(X_train_eda, y_train)

In [ ]:
rf_preds = rf_pipe.predict(X_test_eda)

In [ ]:
print(classification_report(y_test,rf_preds))
#random forest seems to have performed better than the gradient boosted model so far
#I'll try one other gradient boosted model

In [ ]:
lgb = LGBMClassifier()

In [ ]:
lgb_pipe = make_pipeline(scaler,lgb)

In [ ]:
lgb_pipe.fit(X_train_eda, y_train)

In [ ]:
lgb_preds = lgb_pipe.predict(X_test_eda)

In [ ]:
print(classification_report(y_test, lgb_preds))
#it looks like the 1 class is still having trouble being predicted here.
#I'll see if a neural network is any better

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
X_train_scaled = scaler.fit_transform(X_train_eda)
X_test_scaled = scaler.transform(X_test_eda)

In [ ]:
model = Sequential()

In [ ]:
input_shape = X_train_scaled.shape[1]

In [ ]:
def plot_history(history, metric=None):
  fig, axes = plt.subplots(2,1, figsize = (5,10))
  axes[0].plot(history.history['loss'], label = "train")
  axes[0].plot(history.history['val_loss'], label='test')
  axes[0].set_title('Loss')
  axes[0].legend()
  if metric:
    axes[1].plot(history.history[metric], label = 'train')
    axes[1].plot(history.history['val_' + metric], label = 'test')
    axes[1].set_title(metric)
    axes[1].legend()

  plt.show()

In [ ]:
model.add(Dense(12, activation='relu', input_dim=input_shape))

model.add(Dense(10, activation='relu'))

model.add(Dense(5, activation='relu'))

model.add(Dense(1, activation='sigmoid'))
estop = EarlyStopping(patience=10)

In [ ]:
model.compile(optimizer='adam', loss='bce', metrics='acc')

In [ ]:
history = model.fit(X_train_scaled, y_train,
                    validation_data = (X_test_scaled, y_test),
                    epochs = 100,
                    callbacks=[estop])

In [ ]:
plot_history(history, metric='acc')

In [ ]:
nn_preds = model.predict(X_test_eda)

In [ ]:
nn_preds_r = nn_preds.round()

In [ ]:
print(classification_report(y_test, nn_preds_r))

After thorough examination, it seems that the neural network isn't able to predict the positive class either. I'll revisit the LGBMClassifier to see if that performs better with a gridsearch

In [ ]:
lgb_pipe.get_params()

In [ ]:
lgb_params = {'lgbmclassifier__learning_rate':[0.01,0.05,0.1,0.15,0.2,0.3],
              'lgbmclassifier__max_depth':[-1,0,1,2,3,5,8],
              'lgbmclassifier__min_child_samples':[3,5,10,15,20,30],
              'lgbmclassifier__n_estimators':[100,200,300]}

In [ ]:
lgb_grid = GridSearchCV(lgb_pipe,lgb_params)

In [ ]:
#lgb_grid.fit(X_train_eda,y_train)

In [ ]:
#print(lgb_grid.best_params_)

In [ ]:
best_lgb = LGBMClassifier(learning_rate=.01,
                          min_child_samples=5,
                          n_estimators=300)

In [ ]:
best_lgb_pipe = make_pipeline(scaler,best_lgb)

In [ ]:
best_lgb_pipe.fit(X_train_eda,y_train)

In [ ]:
best_lgb_preds = best_lgb_pipe.predict(X_test_eda)

In [ ]:
print(classification_report(y_test,best_lgb_preds))

Slightly better precision for predicting the 1 class, I'll see if random forest does better with hypertuning

In [ ]:
rf_pipe.get_params()

In [ ]:
rf_params = {'randomforestclassifier__max_depth':[1,2,3,5,10,20],
             'randomforestclassifier__min_samples_leaf':[1,3,5,10,15],
             'randomforestclassifier__min_samples_split':[2,3,4,7,10],
             'randomforestclassifier__n_estimators':[100,200,300,500],
             'randomforestclassifier__max_leaf_nodes':[25,50,200,500]}

In [ ]:
#rf_grid = GridSearchCV(rf_pipe,rf_params)

In [ ]:
#rf_grid.fit(X_train_eda,y_train)

In [ ]:
#print(rf_grid.best_params_)

In [ ]:
best_rf = RandomForestClassifier()

In [ ]:
best_rf_pipe = make_pipeline(scaler,best_rf)

In [ ]:
best_rf_pipe.fit(X_train_eda,y_train)

In [ ]:
best_rf_preds = best_rf_pipe.predict(X_test_eda)

In [ ]:
print(classification_report(y_test,best_rf_preds))

It looks like the non-optimized RandomForestClassifier produced the best accuracy. However, GridSearchCV does not determine best_params based on precision and recall together, so I'll try some manual hypertuning.

In [ ]:
best_rf2 = RandomForestClassifier(min_samples_leaf=1,n_estimators=500)

In [ ]:
best_rf_pipe2 = make_pipeline(scaler, best_rf2)

In [ ]:
best_rf_pipe2.fit(X_train_eda,y_train)

In [ ]:
best_rf2_preds = best_rf_pipe2.predict(X_test_eda)

In [ ]:
print(classification_report(y_test, best_rf2_preds))

Seeing if some feature engineering helps at all

In [ ]:
pca = PCA()

In [ ]:
X_train_eda_scaled = scaler.fit_transform(X_train_eda)

In [ ]:
pca.fit(X_train_eda_scaled)

In [ ]:
plt.plot(range(1,28), pca.explained_variance_ratio_[:27], marker='x')
plt.xticks(ticks = range(1,28), rotation=90)
plt.xlabel('Principal Component')
plt.ylabel('Proportion of Variance Explained');

In [ ]:
#it looks like there might be a few good options for how many principle components to include
#I will start with 5 and go up from there if 5 features does not seem to be enough

In [ ]:
X_test_eda_scaled = scaler.fit_transform(X_test_eda)

In [ ]:
pca = PCA(n_components=5)
X_train_pca = pca.fit_transform(X_train_eda_scaled)
X_test_pca = pca.fit_transform(X_test_eda_scaled)

In [ ]:
best_rf2.fit(X_train_pca,y_train)

In [ ]:
best_rf2_preds_pca = best_rf2.predict(X_test_pca)

In [ ]:
print(classification_report(y_test, best_rf2_preds_pca))

Looks like 5 features isn't quite enough. I'll try PCA from the next "elbow" in the line plot above.

In [ ]:
pca2=PCA(n_components=9)
X_train_pca2 = pca2.fit_transform(X_train_eda_scaled)
X_test_pca2 = pca2.fit_transform(X_test_eda_scaled)

In [ ]:
best_rf2.fit(X_train_pca2,y_train)

In [ ]:
best_rf2_preds_pca2=best_rf2.predict(X_train_pca2)

In [ ]:
print(classification_report(best_rf2_preds_pca2,y_test))